<a href="https://colab.research.google.com/github/abhinavarorags/CoolStuff/blob/test/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ARIMA
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Load dataset
def load_data(file_path, url='https://raw.githubusercontent.com/abhinavarorags/CoolStuff/refs/heads/test/sample_data.csv'):
    # Load dataset
    try:
        data = pd.read_csv(file_path)
    except FileNotFoundError:
        # If file not found, download from URL
        data = pd.read_csv(url)
    return data

# Load dataset
file_path = '/mnt/data/sample_data.csv'
data = load_data(file_path)

# Prepare the time series based on 'Close' and 'Open' difference
reward_series = data['Close'] - data['Open']  # Reward is positive if 'Close' is greater than 'Open'

# Split data into training and testing
train_size = int(len(reward_series) * 0.8)
train, test = reward_series[:train_size], reward_series[train_size:]

# Fit ARIMA model
from itertools import product

# Tune ARIMA parameters using grid search
p = range(1, 6)
d = range(0, 2)
q = range(0, 3)

best_order = None
best_aic = float('inf')

for params in product(p, d, q):
    try:
        temp_model = ARIMA(train, order=params)
        temp_model_fit = temp_model.fit()
        if temp_model_fit.aic < best_aic:
            best_aic = temp_model_fit.aic
            best_order = params
    except:
        continue

order = best_order  # Best p, d, q parameters for ARIMA
model = ARIMA(train, order=order)
model_fit = model.fit()
print(model_fit.summary())

# Forecast
forecast = model_fit.forecast(steps=len(test))

# Generate Buy/Sell/Hold recommendations based on forecast
def generate_recommendations(forecast, threshold=0.1):
    recommendations = []
    for value in forecast:
        if value > threshold:
            recommendations.append("Buy Long")
        elif value < -threshold:
            recommendations.append("Buy Long")
        elif value < 0:
            recommendations.append("Sell Short")
        else:
            recommendations.append("Hold")
    return recommendations

recommendations = generate_recommendations(forecast)

# Print recommendations summary
correct_predictions = {"Buy Long": 0, "Sell Short": 0, "Hold": 0}
wrong_predictions = {"Buy Long": 0, "Sell Short": 0, "Hold": 0}

for i in range(len(test)):
    actual = test.iloc[i]
    predicted_action = recommendations[i]
    if (predicted_action == "Buy Long" and actual > 0) or (predicted_action == "Sell Short" and actual < 0) or (predicted_action == "Hold" and actual == 0):
        correct_predictions[predicted_action] += 1
    else:
        wrong_predictions[predicted_action] += 1

print("Test Data saved with recommendations")
print("\t\tCorrect Predictions  Wrong Predictions")
for action in ["Buy Long", "Sell Short", "Hold"]:
    print(f"{action}\t\t{correct_predictions[action]}\t\t\t{wrong_predictions[action]}")

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(train.index, train, label='Training Data')
plt.plot(test.index, test, label='Actual Test Data', color='green')
plt.plot(test.index, forecast, label='ARIMA Forecast', color='red')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('ARIMA Forecast vs Actual')
plt.legend()
plt.show()

# Calculate accuracy metrics
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

test_mape = mean_absolute_percentage_error(test, forecast)
print(f"Mean Absolute Percentage Error (MAPE) on Test Data: {test_mape:.2f}%")


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
